In [1]:
import abc
import numpy as np
import os
import json
from openbabel import pybel
from rdkit import Chem
from rdkit.Chem import AllChem
import pdb
import itertools
from tqdm import tqdm
from Features import DualMol
from Features import Featurizer
import shutil

# Configuration
First step is to set up the configuration file.

The code Below does this for you assuming this jupyter notebook is run in the same directory as all of the other 

In [12]:
config_values = {
"training_set_directory": os.path.join(os.getcwd(),"training_sets"), # directory where all new training sets will be made
"searches_directory": os.getcwd(),  # Directory where the reactions.json file is found
"species_db":os.path.join(os.getcwd(),os.path.join('dft_results','dft_data.json')), # path to the file with the post processesd DFT data 
"dft_results": os.path.join(os.getcwd(),'dft_results'), # path to the directory of the relaxed xyz files of molecules
"molecular_dict":  os.path.join(os.getcwd(),'molec_descriptor.dict') #json databaseof all features
}

json.dump(config_values,open('config.json','w'),indent=2)

In [13]:
if "training_sets" not in os.listdir():
    os.mkdir("training_sets")

The way the code works is as follows:

The Featurizer object will be instantiated with all the necessary  paths. It will first update the molecular dictionary, containing the molecular features/descriptors. The source from the update comes from two separate locations:

The first is the relaxed structure of the molecule, which should be located inside a directory within the "dft_results" directory. Note that the file containing the relaxed/optimized specie should be and XYZ file and have the "relaxed_" prefix. Currently xyz files are the only ones supported, but it should not be dificult to make the code robust to other file types 

        Example: dft-results/XXXXXXXXXXXXXX-UHFFFAOYSA-N/relaxed_XXXXXXXXXXXXXX-UHFFFAOYSA-N.xyz

The second Source of data is the dft_data.json file which has 4 values for each specie:

    - Gibbs free energy at 0K (G0) [J/mol]
    
    - Difference between Gibbs free energy at 0K and Gibbs free energy at 300K  (dG300) [J/mol]
    
    - Highest Occupied Molecular Orbital energy Value [eV]
    
    - Lowest Unoccupied Molecular Orbital energy Value [eV]
    
    We have also added spin dependent 

With all of this in place we can create a training set with all of the available features. Features can be included and omitted  to create training sets of different features. The created training values and feature vectors will be placed in the training_sets directory, unless otherwise specified in the config file.

In [50]:
my_featurizer = Featurizer()
molecdict = my_featurizer.update_molecular_dict(out=True)
my_featurizer.trainingsetgenerator(features = [
    "gibbs",
    "entropy",
    "topo",
    "morgan",
    "hom-lum",
    "homo",
    "lumo",
    'min_lumo_reactants',
    'max_lumo_reactants',
    'max_h-l_reactants',
    'min_homo_reactants',
    'max_homo_reactants',
    'min_lumo_products',
    'max_lumo_products',
    'max_h-l_products',
    'min_homo_products',
    'max_homo_products',
],out=True)

  0%|                                                                                          | 0/6063 [00:00<?, ?it/s]==============================
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

  3%|██▏                                                                            | 172/6063 [00:00<00:20, 282.32it/s][12:46:10] Explicit valence for atom # 1 N, 4, is greater than permitted
[12:46:10] Explicit valence for atom # 1 N, 4, is greater than permitted
  4%|██▉                                                                            | 230/6063 [00:00<00:20, 284.41it/s]

couldn't add Hydrogens, implicit hydrogens might be missing


*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

  6%|████▉                                                                          | 375/6063 [00:01<00:20, 279.59it/s][12:46:10] Explicit valence for atom # 1 N, 4, is greater than permitted
[12:46:10] Explicit valence for atom # 1 N, 4, is greater than permitted
  7%|█████▋                                                                         | 434/6063 [00:01<00:19, 284.55it/s]

couldn't add Hydrogens, implicit hydrogens might be missing


  8%|██████                                                                         | 463/6063 [00:01<00:19, 282.27it/s]==============================
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

 10%|███████▉                                                                       | 610/6063 [00:02<00:18, 287.27it/s]

...

100%|██████████████████████████████████████████████████████████████████████████████| 6063/6063 [00:21<00:00, 281.56it/s]


full info molecules 6017


28953it [01:45, 273.81it/s]


{'gibbs': 0, 'entropy': 1, 'topo': 7, 'morgan': 107, 'hom-lum': 108, 'homo': 109, 'lumo': 110, 'min_lumo_reactants': 111, 'max_lumo_reactants': 112, 'max_h-l_reactants': 113, 'min_homo_reactants': 114, 'max_homo_reactants': 115, 'min_lumo_products': 116, 'max_lumo_products': 117, 'max_h-l_products': 118, 'min_homo_products': 119, 'max_homo_products': 120}


[array([[-4.86499346e+02,  4.94403605e+01, -2.92528740e-01, ...,
          8.27305000e+00, -6.99058000e+00, -6.99058000e+00],
        [-4.55574540e+02,  4.44152860e+01, -9.61331711e-01, ...,
          5.08333000e+00, -5.96429000e+00, -5.96429000e+00],
        [-4.51263156e+02,  4.77067035e+01, -1.16350396e+00, ...,
          5.07604000e+00, -5.57917000e+00, -5.57917000e+00],
        ...,
        [-2.37001211e+01,  1.80066977e+00, -7.53336385e-01, ...,
          1.19714000e+00, -3.78326000e+00, -3.78326000e+00],
        [-2.74155212e+02,  5.55656932e+01,  3.37644960e-01, ...,
          3.69964000e+00, -5.15203000e+00, -5.15203000e+00],
        [-1.91907661e+02,  3.18152448e+01,  1.81862582e+00, ...,
          1.47009000e+00, -5.72516000e+00, -5.72516000e+00]]),
 array([22133.4  ,   157.737,  -111.713, ..., 71175.566, -2093.399,
         2590.   ]),
 array([    0,     6,     7, ..., 28949, 28951, 28952])]

Once the cell above is done running the "training_set" directory will have a '.trainingvalues' and a '.trainingfeatures' file that can be used read with the np.load function to read in an x array of descriptors and a y array of activatione energy values. Additionally a 'reaction_indices_out.json' file is present which gives the index of the reaction back in the reactions.json file 

i.e. 

    the first element reaction_indices_out.json says '6', meaning that the first element of xxx.trainvalues and xxx.trainfeatures refers to reaction no. 6 in reactions.json.

Lastly the features_explained.json gives an index value for all the indices in the descriptor of the reaction explaining what property it corresponds to.

# How to add new molecules and reactions

To add new entries to the reaction datase you require access to a DFT package and the reaction database. This notebook merely provides the steps needed once the DFT values are calculated.
In order to add new data to the training set you need the following: 

    - Activation energy and stoichiometry for reactions to be added (All values used were taken from the RMG database but actuvation energies can also be calculated using Transition state search methods).

    - Geometry Optimized/Relaxed Molecular structure 
    
    - Energetics from geometry relaxation and Thermodynamic calculation (Gibbs energies @ 0 and 300 K, HOMO and LUMO)
    
    NB: Current thermodynamic  data is calculated  using PBE with def2-SVP basis sets in TURBOMOLE. To add new species to this specific dataset the same XC-Functional and basis set must be used (ideally with the same DFT package).
    link to Turbomole manual : https://www.turbomole.org/wp-content/uploads/2019/10/Turbomole_Manual_7-4.pdf
    Page (44/270 has instructions on how it can be used) In all other cases the thermodynamic data for all molecules must be recalculated and included. 

The geometry optimized molecule coordinate file in XYZ format must be placed as follows: 
         
         Example: package_installation_path/dft-results/XXXXXXXXXXXXXX-UHFFFAOYSA-N/relaxed_XXXXXXXXXXXXXX-UHFFFAOYSA-N.xyz

Then, the DFT information has to be updated in the dft_data.json file as mentioned above. This file has different values that must be filled in, namely:

    - "G0":float((XXX.XXX)  # Free energy at 0 K in KJ/mol
    
    - "dG300": float(XXX.XXX) , # Change in free energy between 0 and 300 K G@300-G0 in KJ/mol
    
    - "HOMO": float((XXX.XXX) , # Value of HOMO of molecule in eV
    
    - "LUMO": float((XXX.XXX), # Value of LUMO of molecule in eV
    



Once again, note that this notebook will not calculate the DFT neccessary information, this must be done with an external package. In principle any DFT package can calculate the neccessary values. We recommend TURBOMOLE for the use of their  "freeh" property calculation package that provides the neccessary free energy values.

Once all molecules have been placedThe reaction then has to be included in the file reactions.data where the inchi keys of the products and reactions must be filled in as follows:


"ProdInChI": [
      "XXXXXXXXXXXXXX-UHFFFAOYSA-N"
      "XXXXXXXXXXXXXX-UHFFFAOYSA-N"
    ],
    "ReacInChI": [
      "XXXXXXXXXXXXXX-UHFFFAOYSA-N",
      "XXXXXXXXXXXXXX-UHFFFAOYSA-N"
    ]


and the activation energy must be filled in as follows:
    
    'Ea [kJ/mol]': float(XXX.XXX)


with supported units being :
    
    '[J/mol]':
    
    '[cal/mol]'
    
    '[kJ/mol]'
    
    '[kcal/mol]'


# Worked Out Example

We now show how to add the reaction 
Si + H4Si -> H4Si2

In [46]:
# From DFT calculations we calculated the following values : 
new_molecule_data = {
  "XUIMIQQOPSSXEZ-UHFFFAOYSA-N": {
    "G0": -759345.7427262537,
    "dG300": -42.291110149701126,
    "HOMO": -4.58985,
    "LUMO": -2.96025,
  },
    "BLRPTPMANUNPDV-UHFFFAOYSA-N": {
    "G0": -765831.3345236926,
    "dG300": -57.260016584652476,
    "HOMO": -8.4915,
    "LUMO": 0.21139,
  },
"BAGSMSWGHZVCIQ-UHFFFAOYSA-N": {
    "G0": -1525368.9849108877,
    "dG300": -67.73588191042654,
    "HOMO": -5.72516,
    "LUMO": -4.25507,
  }
}

In [47]:
#We take from the NIST database the following information
new_reaction = {
    "Reactants":[
      "InChI=1S/Si",
      "InChI=1S/H4Si/h1H4"
    ],
    "Products": [
      "InChI=1S/H4Si2/c1-2/h1H,2H3"
    ],
    "Ea [kJ/mol]": 2.59,
    "link": "https://kinetics.nist.gov/kinetics/ReactionSearch?r0=7440213&r1=7803625&r2=0&r3=0&r4=0&p0=50420901&p1=0&p2=0&p3=0&p4=0&expandResults=true&",
    "ReacInChI": [
      "XUIMIQQOPSSXEZ-UHFFFAOYSA-N",
      "BLRPTPMANUNPDV-UHFFFAOYSA-N"
    ],
    "ProdInChI": [
      "BAGSMSWGHZVCIQ-UHFFFAOYSA-N"
    ],
    "rxn_index": 28952
}

In [48]:
# Now we incorporate all the information: 

all_reactions = json.loads(open("reactions.json",'r').read())
all_reactions.append(new_reaction)
json.dump(all_reactions, open('reactions.json','w'), indent=2)

all_mol_data = json.loads(open("./dft_results/dft_data.json",'r').read())
all_mol_data.update(new_molecule_data)
json.dump(all_mol_data, open('./dft_results/dft_data.json','w'), indent=2)

if not os.path.isdir('./dft_results/XUIMIQQOPSSXEZ-UHFFFAOYSA-N'):
    os.mkdir('./dft_results/XUIMIQQOPSSXEZ-UHFFFAOYSA-N')
molec = pybel.readstring('inchi',"InChI=1S/Si")
molec.localopt()
open('./dft_results/XUIMIQQOPSSXEZ-UHFFFAOYSA-N/relaxed_XUIMIQQOPSSXEZ-UHFFFAOYSA-N.xyz','w').write(molec.write('xyz'))

if not os.path.isdir('./dft_results/BLRPTPMANUNPDV-UHFFFAOYSA-N'):
    os.mkdir('./dft_results/BLRPTPMANUNPDV-UHFFFAOYSA-N')
molec = pybel.readstring('inchi',"InChI=1S/H4Si/h1H4")
molec.localopt()
open('./dft_results/BLRPTPMANUNPDV-UHFFFAOYSA-N/relaxed_BLRPTPMANUNPDV-UHFFFAOYSA-N.xyz','w').write(molec.write('xyz'))

if not os.path.isdir('./dft_results/BAGSMSWGHZVCIQ-UHFFFAOYSA-N'):
    os.mkdir('./dft_results/BAGSMSWGHZVCIQ-UHFFFAOYSA-N')
molec = pybel.readstring('inchi',"InChI=1S/H4Si2/c1-2/h1H,2H3")
molec.localopt()
open('./dft_results/BAGSMSWGHZVCIQ-UHFFFAOYSA-N/relaxed_BAGSMSWGHZVCIQ-UHFFFAOYSA-N.xyz','w').write(molec.write('xyz'))



297